In [26]:
### Import libraries
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from datetime import date
from datetime import time
from datetime import datetime

import seaborn as sns #charts
sns.set()

import folium #mapping
SF_COORDINATES = (37.76, -122.45)

In [53]:
df_prediction = pd.read_csv('/home/ec2-user/SageMaker/business_id_predictions.csv', sep = ',', index_col = 'business_id')  
df_prediction.head(3)

# I have to drop the column because of how I saved the master data set 

,score,predictions
business_id,,
19,B,B
24,A,A
31,B,B


In [54]:
df_name = pd.read_csv('/home/ec2-user/SageMaker/df_location_master_final.csv', sep = ',') #, index_col = 'business_id') 
df_name = df_name.drop(columns=['Unnamed: 0'])

df_name = df_name[['business_id', 'business_name', 'business_latitude', 'business_longitude', 'neighborhood']]
df_name.head(3)

,business_id,business_name,business_latitude,business_longitude,neighborhood
0,852,New Hollywood Bakery & Restaurant,37.797083,-122.406057,North Beach/Chinatown
1,852,New Hollywood Bakery & Restaurant,37.797083,-122.406057,North Beach/Chinatown
2,852,New Hollywood Bakery & Restaurant,37.797083,-122.406057,North Beach/Chinatown


In [55]:
# Trick to group by was to drop duplicates
df_org = df_name.drop_duplicates(subset=['business_id'])
df_org

,business_id,business_name,business_latitude,business_longitude,neighborhood
0,852,New Hollywood Bakery & Restaurant,37.797083,-122.406057,North Beach/Chinatown
7,2934,Lou's Fish Shack,37.808126,-122.417901,North Beach/Chinatown
21,4323,DIVA'S,37.786627,-122.419493,Polk/Russian Hill (Nob Hill)
30,5281,Eva's Catering,37.810119,-122.417716,Sunset
46,5312,The Waterfront,37.800791,-122.396553,75 Howard Area
...,...,...,...,...,...
48708,71783,Imagine Culinary Group,37.771364,-122.402279,South of Market
48719,71823,Subway Sandwiches #24254,37.792237,-122.400124,NoName-94104
48730,71841,"Rapha Racing, LLC",37.798078,-122.435444,Marina
48737,71909,Shanghai Dumpling King 2,37.731568,-122.450970,St. Francis Wood/Miraloma/West Portal


In [56]:
print(df_org.shape)
print(df_prediction.shape)

# The difference is the further processing that was dine in R, mostly nulls. I did think of this and always used my primary key businessTo match at business_id

(5396, 5)
(4996, 2)


In [59]:
df_predicted_restaurant_score=pd.merge(df_org, df_prediction, on='business_id', how='inner')
df_predicted_restaurant_score.head(7)

,business_id,business_name,business_latitude,business_longitude,neighborhood,score,predictions
0,852,New Hollywood Bakery & Restaurant,37.797083,-122.406057,North Beach/Chinatown,D,D
1,2934,Lou's Fish Shack,37.808126,-122.417901,North Beach/Chinatown,C,C
2,4323,DIVA'S,37.786627,-122.419493,Polk/Russian Hill (Nob Hill),B,B
3,5281,Eva's Catering,37.810119,-122.417716,Sunset,D,D
4,5312,The Waterfront,37.800791,-122.396553,75 Howard Area,A,A
5,5313,S.F. Bay Concessionaires Commissary,37.809474,-122.411851,North Beach/Chinatown,A,A
6,5409,KIEM'S SEAFOOD,37.780161,-122.413779,Hayes Valley/Tenderloin/North of Market,A,A


In [65]:
df_D = df_predicted_restaurant_score.loc[df_predicted_restaurant_score['predictions'] == 'D']
df_D.head(7)

,business_id,business_name,business_latitude,business_longitude,neighborhood,score,predictions
0,852,New Hollywood Bakery & Restaurant,37.797083,-122.406057,North Beach/Chinatown,D,D
3,5281,Eva's Catering,37.810119,-122.417716,Sunset,D,D
31,37879,Cafe Mystique,37.761482,-122.435250,Castro/Noe Valley,D,D
40,63286,El Paraiso,37.751146,-122.413055,Inner Mission/Bernal Heights,D,D
44,66522,Java House LLC,37.781720,-122.386312,Potrero Hill,D,D
50,67920,Noori,37.763624,-122.478716,Sunset,D,D
53,68148,Sheng Kee Bakery & Cafe,37.764232,-122.466705,Sunset,D,D


In [66]:
df_D_C = df_predicted_restaurant_score.loc[df_predicted_restaurant_score['predictions'].isin(['D', 'C'])]
df_D_C.head(7)

,business_id,business_name,business_latitude,business_longitude,neighborhood,score,predictions
0,852,New Hollywood Bakery & Restaurant,37.797083,-122.406057,North Beach/Chinatown,D,D
1,2934,Lou's Fish Shack,37.808126,-122.417901,North Beach/Chinatown,C,C
3,5281,Eva's Catering,37.810119,-122.417716,Sunset,D,D
27,28169,Bloomingdales,37.784123,-122.406377,South of Market,C,C
29,36519,Pop's Sandwich Shop,37.743940,-122.453610,St. Francis Wood/Miraloma/West Portal,C,C
31,37879,Cafe Mystique,37.761482,-122.435250,Castro/Noe Valley,D,D
32,59152,Patio Cafe,37.760351,-122.434810,Castro/Noe Valley,C,C


In [67]:
map = folium.Map(location=SF_COORDINATES, zoom_start=12) 
MAX_RECORDS = 50 
for each in df_D[0:MAX_RECORDS].iterrows():
    folium.Marker([each[1]["business_latitude"],each[1]["business_longitude"]],
        popup=each[1]["business_name"]+ each[1]["neighborhood"]).add_to(map)
map